In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
import shutil
from IPython.display import Image, clear_output  # to display images


In [ ]:
!pip install roboflow

# might need to be modified before run
os.environ["DATASET_DIRECTORY"] = "/content/datasets" # path to datasets directory
yolov5_directory = '/content/yolov5' # path to `yolov5` directory (*contains train.py and val.py)
os.environ["YOLOV5_DIRECTORY"] = yolov5_directory

from roboflow import Roboflow
rf = Roboflow(api_key=#API#)
project = rf.workspace().project(#PROJECTNAME#)
dataset = project.version(1).download("yolov5")

In [ ]:
# change model here
yolov5_model = 'n' #@param ["n", "s", "m", "l", "x"]
config = 'yolov5' + yolov5_model + '.yaml'
startWeights = 'yolov5' + yolov5_model + '.pt'

os.environ["YOLO_CONFIG"] = config
os.environ["INIT_WEIGHTS"] = startWeights

In [ ]:
!python train.py --img 416 --batch-size -1 --epochs 300 --data {dataset.location}/data.yaml --cfg {config} --weights {startWeights} --cache

In [ ]:
def getLastCreatedExp():
  """
    Returns name of last created exp folder (located in /runs/train/...)
  """
  contents = os.listdir(f'{yolov5_directory}/runs/train')
  contentsWithoutZIP = []
  for f in contents:
    if '.zip' not in f and 'exp' in f:
      contentsWithoutZIP.append(f)
  contentsWithoutZIP.sort()
  return contentsWithoutZIP[-1]

last_exp = getLastCreatedExp()

In [ ]:
# evaluates trained model on test set
!python val.py --data {dataset.location}/data.yaml --batch-size 8 --weights {yolov5_directory}/runs/train/{last_exp}/weights/last.pt

Weights and train results are located in `runs/train/exp*` whiel test results are saved in `runs/val/exp*`.

